<h1 align="center">DotNetStac</h1>
<h2 align="center">
.Net library for working with Spatio Temporal Asset Catalogs (<a href="https://stacspec.org">STAC</a>)
</h2>

### **DotNetStac** helps you to work with [STAC](https://stacspec.org) ([catalog](https://github.com/radiantearth/stac-spec/tree/master/catalog-spec), [collection](https://github.com/radiantearth/stac-spec/tree/master/collection-spec), [item](https://github.com/radiantearth/stac-spec/tree/master/catalog-spec))
In a nutshell, the library allows import/export of STAC JSON documents (Serialization/Deserialization using [Newtonsoft.JSON](https://www.newtonsoft.com/json)) to typed object with properties represented in enhanced objects such as geometries, time stamp/period/span, numerical values and many more via STAC extension plugins engine.

## Getting Started

In this notebook, we are going to review the library's features through a series of code examples.

#### Important Note about .Net programming and C# language usage in this library

We chose to design and implement the objects of this library as plain as possible (e.g. POCO) in order to keep the structured and typed nature of the C# language. So, we will deal with normal classes, without more attributes describing infrastructure concerns or other responsibilities that your domain objects shouldn't have.

### 1. Install DotNetStac
First, we either install lastest version of DotNetStac in the notebook or  we use the locally built binaries

In [ ]:
// Use Nuget Package
#r "nuget:DotNetStac,0.3.1"

// Or use local package
//#i "nuget:/tmp/DotNetStac/src/DotNetStac/bin/Debug/"
//#r "nuget:DotNetStac,0.3.1"

### 2. Deserialization of STAC documents

The serialization engine used is [Newtonsoft.JSON](https://www.newtonsoft.com/json)) supporting current and older versions of the specifications.

Let's start reading a STAC catalog online

In [ ]:
using Stac;

// StacFactory.Load is the helper to start loading any STAC document
IStacCatalog catalog = (IStacCatalog)StacFactory.Load("https://cbers-stac-1-0.s3.amazonaws.com/CBERS4/catalog.json");

Console.Out.WriteLine(catalog.Id + ": " + catalog.Description);
Console.Out.WriteLine(catalog.StacVersion);


### 3. Navigation from a Catalogue

Embedded navigation methods allows to seamlessly traverse a STAC catalog through collections and items.

Using the previously loaded catalog, let's print a tree of resources (children, items, assets)

In [ ]:
using System.Linq;

public static void ListChildrensItemsAndAssets(IStacCatalog catalog, string prefix = "", int limit = 2)
{
    // Get children first (sub catalogs and collections)
    foreach (var child in catalog.GetChildren().Values.Take(limit))
    {
        Console.Out.WriteLine(prefix + child.Id + ": " + child.Description);

        foreach(var item in child.GetItems().Values.Take(limit)){
            Console.Out.WriteLine(prefix + " " + item.Id);
            foreach(var asset in item.Assets.Values){
                Console.Out.WriteLine(prefix + " *[" + asset.MediaType + "] " + asset.Uri);
            }
        }

        ListChildrensItemsAndAssets(child, prefix + " ");
    }
}

ListChildrensItemsAndAssets(catalog);

### 4.Creation and serialization of STAC documents

Let's now create a STAC collection programmatically using the the plain Collection object

In [ ]:
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;
using Stac;
using Stac.Catalog;
using Stac.Collection;
using System;
using System.Collections.Generic;


  // First the mandatory elements of a Collection
  // Spatial Extent
  StacExtent extent = new StacExtent();
  extent.Spatial = new StacSpatialExtent(-180, -56, 180, 83);
  // Temporal Extent
  extent.Temporal = new StacTemporalExtent(DateTime.Parse("2015-06-23T00:00:00Z").ToUniversalTime(), null);

  // Create the Collection
  StacCollection collection = new StacCollection("COPERNICUS/S2",
    "Sentinel-2 is a wide-swath, high-resolution, multi-spectral\nimaging mission supporting Copernicus Land Monitoring studies,...",
                                      extent);

  // Title
  collection.Title = "Sentinel-2 MSI: MultiSpectral Instrument, Level-1C";

  // Links (self, parent, root, other) via different helpers
  collection.Links.Add(StacLink.CreateSelfLink(new Uri("https://storage.cloud.google.com/earthengine-test/catalog/COPERNICUS_S2.json")));
  collection.Links.Add(StacLink.CreateParentLink(new Uri("https://storage.cloud.google.com/earthengine-test/catalog/catalog.json")));
  collection.Links.Add(StacLink.CreateRootLink(new Uri("https://storage.cloud.google.com/earthengine-test/catalog/catalog.json")));
  collection.Links.Add(new StacLink(new Uri("https://scihub.copernicus.eu/twiki/pub/SciHubWebPortal/TermsConditions/Sentinel_Data_Terms_and_Conditions.pdf"), "license", "Legal notice on the use of Copernicus Sentinel Data and Service Information", null));

  // Keywords
  collection.Keywords = new System.Collections.ObjectModel.Collection<string>(new string[] {
      "copernicus",
      "esa",
      "eu",
      "msi",
      "radiance",
      "sentinel"});

  // Collection of Providers
  collection.Providers = new System.Collections.ObjectModel.Collection<StacProvider>(
      new StacProvider[]{new StacProvider("European Union/ESA/Copernicus"){
          Roles = new List<StacProviderRole>() { StacProviderRole.producer, StacProviderRole.licensor},
          Uri = new Uri("https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi")
  }});

  // Summaries
  // Stat Summary for the dates
  collection.Summaries.Add("datetime",
      new StacSummaryStatsObject<DateTime>(
          DateTime.Parse("2015-06-23T00:00:00Z").ToUniversalTime(),
          DateTime.Parse("2019-07-10T13:44:56Z").ToUniversalTime()
      )
  );

  // Value Set Summary for the platforms
  collection.Summaries.Add("platform",
      new StacSummaryValueSet<string>(new string[] { "sentinel-2a", "sentinel-2b" })
  );

  collection.Summaries.Add("constellation",
      new StacSummaryValueSet<string>(new string[] { "sentinel-2" })
  );

  collection.Summaries.Add("instruments",
      new StacSummaryValueSet<string>(new string[] { "msi" })
  );

  // Stat Summary for specific extensions
  collection.Summaries.Add("view:off_nadir",
      new StacSummaryStatsObject<double>(
          0.0,
          100
      )
  );

  collection.Summaries.Add("view:sun_elevation",
      new StacSummaryStatsObject<double>(
          6.78,
          89.9
      )
  );

  collection.Summaries.Add("sci:citation",
      new StacSummaryValueSet<string>(new string[] { "Copernicus Sentinel data [Year]" })
  );

  collection.Summaries.Add("gsd",
      new StacSummaryValueSet<int>(new int[] {
          10,
          30,
          60
      })
  );

  collection.Summaries.Add("proj:epsg",
      new StacSummaryValueSet<int>(new int[]
          { 32601,32602,32603,32604,32605,32606,32607,32608,32609,32610,32611,32612,32613,32614,32615,32616,32617,32618,32619,32620,32621,32622,32623,32624,32625,32626,32627,32628,32629,32630,32631,32632,32633,32634,32635,32636,32637,32638,32639,32640,32641,32642,32643,32644,32645,32646,32647,32648,32649,32650,32651,32652,32653,32654,32655,32656,32657,32658,32659,32660}
      )
  );

  // Value Set Summary can also be set using JToken objects from Newtonsoft.Json library
  collection.Summaries.Add("eo:bands",
      new StacSummaryValueSet<JObject>(new JObject[] {
          new JObject {
              { "name", "B1" },
              { "common_name", "coastal" },
              { "center_wavelength", 4.439 }
          },
          new JObject {
              { "name", "B2"},
              { "common_name", "blue"},
              { "center_wavelength", 4.966}
          },
          new JObject {
              { "name", "B3"},
              { "common_name", "green"},
              { "center_wavelength", 5.6}
          },
          new JObject {
              { "name", "B4"},
              { "common_name", "red"},
              { "center_wavelength", 6.645}
          },
          new JObject {
              { "name", "B5"},
              { "center_wavelength", 7.039}
          },
          new JObject {
              { "name", "B6"},
              { "center_wavelength", 7.402}
          },
          new JObject {
              { "name", "B7"},
              { "center_wavelength", 7.825}
          },
          new JObject {
              { "name", "B8"},
              { "common_name", "nir"},
              { "center_wavelength", 8.351}
          },
          new JObject {
              { "name", "B8A"},
              { "center_wavelength", 8.648}
          },
          new JObject {
              { "name", "B9"},
              { "center_wavelength", 9.45}
          },
          new JObject {
              { "name", "B10"},
              { "center_wavelength", 1.3735}
          },
          new JObject {
              { "name", "B11"},
              { "common_name", "swir16"},
              { "center_wavelength", 1.6137}
          },
          new JObject {
              { "name", "B12"},
              { "common_name", "swir22"},
              { "center_wavelength", 2.2024}
          }
      })
  );

  // Serialize
  var json = JsonConvert.SerializeObject(collection, Formatting.Indented);
  // Print JSON!
  Console.WriteLine(json)

### 5. STAC Extensions support [experimental]

The library implements STAC extensions with a "plugins" engine system for embedding extension related functions (e.g. sat: orbit file download, sar: interferometric search, eo: calibration parameters)

In [ ]:
using System.IO;
using Stac.Extensions.Sat;

// Local File loading
var s1_1_filepath = Path.GetFullPath("files/S1A_IW_SLC__1SDV_20150305T051937_20150305T052005_004892_006196_ABBB.json", Directory.GetCurrentDirectory());
var s1_2_filepath = Path.GetFullPath("files/S1A_IW_SLC__1SDV_20150317T051938_20150317T052005_005067_0065D5_B405.json", Directory.GetCurrentDirectory());
IStacItem sentinel1Item_1 = (IStacItem)StacFactory.Load(s1_1_filepath);
IStacItem sentinel1Item_2 = (IStacItem)StacFactory.Load(s1_2_filepath);

// Load Satellite extension
SatStacExtension satExtension_1 = sentinel1Item_1.GetExtension<SatStacExtension>();
SatStacExtension satExtension_2 = sentinel1Item_2.GetExtension<SatStacExtension>();


Console.Out.WriteLine(string.Format("{0} track: {1} {2}", sentinel1Item_1.GetTitle(),
                                                            satExtension_1.RelativeOrbit, 
                                                            satExtension_1.OrbitState));
Console.Out.WriteLine(string.Format("{0} track: {1} {2}", sentinel1Item_2.GetTitle(),
                                                            satExtension_2.RelativeOrbit, 
                                                            satExtension_2.OrbitState));

#### 5.1 Load Orbital State Vectors

In the next code snippet, we will use the [Satellite Extension](https://github.com/radiantearth/stac-spec/tree/master/extensions/sat) plugin to perform an perpendicular baseline calculation between 2 Sentinel-1 SAR SLC items

![orbital elements](https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Orbit1.svg/290px-Orbit1.svg.png "Orbital Elements")


The purpose of this example is to show the capacity building of the plugin engine to provide with embedded functions out of the box when using STAC catalogs

In [ ]:
satExtension_1.LoadOrbitStateVectors();
satExtension_2.LoadOrbitStateVectors();

satExtension_1.OrbitStateVectors

#### 5.2 Calculate Perpendicular Baseline

Now we ask the Satellite Extension to calculate the Baseline for our 2 Satellite items

<img src="https://slideplayer.com/slide/8654127/26/images/39/Perpendicular+Baseline.jpg" width="400">


In [ ]:
var baseline = satExtension_1.CalculateBaseline(satExtension_2);
display(baseline);

### Other features to come

* STAC API Client
* Importers from other spatio temporal domains (e.g. OGC O&M, Earth Observation profile…)